# Análise Detalhada da Relação Taxa Amostral x Janela x F1-score

A ideia é fazer uma análise do <u>impacto dos parâmetros de Taxa Amostral e Janela</u> na qualidade dos modelos binários. 

Os dados utilizados neste estudo foram os obtidos no `estudo 17`.

Para isto, vamos dividir a análise em dois indicadores:

1. Melhor Taxa Amostral global e a relação de melhores Janelas por Aparelho;
2. Melhores combinações de Taxa Amostral e Janelas por Aparelho.

# Configurações

In [1]:
import os
import sys
import gc
from pprint import pprint
from collections import Counter
import copy
import warnings
warnings.filterwarnings(action="ignore")

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from IPython.display import display
%load_ext watermark

# TODO: implementar rotina na classe PyNILM.utils
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

def listar_variaveis_memoria(ambiente):
    print("* Variáveis instanciadas em memória:")
    print("---")
    total = 0
    for name, size in sorted(((name, sys.getsizeof(value)) for name, value in ambiente.items()),
                             key= lambda x: -x[1])[:10]:
        total += size
        print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))
    print("---")
    print("Total:", sizeof_fmt(total))
    
# TODO: implementar na classe utils
def highlight_col(x):
    r = 'background-color: #D9D9D9'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, -2] = r
    return df1   

In [2]:
# CONSTANTES FUNDAMENTAIS DE ORGANIZACAO DE PASTAS/ARQUIVOS

# Path do arquivo H5 (base REDD ja preparada p/ NILMTK) e outros insumos fundamentais
caminho_dados = "D:/Projetos/phd-thesis/datasets/"

# Definir diretorios onde iremos salvar os insumos gerados do notebook (dados, imagens, etc.)
caminho_dados_notebook = os.path.join(caminho_dados, "19") # Num. notebook
if not os.path.isdir(caminho_dados_notebook):
    os.makedirs(caminho_dados_notebook)
caminho_imagens_notebook = os.path.join(caminho_dados_notebook, "imagens") # Num. notebook
if not os.path.isdir(caminho_imagens_notebook):
    os.makedirs(caminho_imagens_notebook)

# Path do arquivo H5 (base REDD ja preparada p/ NILMTK)
caminho_redd = os.path.join(caminho_dados, "REDD/low_freq")

# Path completo do arquivo REDD
arquivo_dataset = os.path.join(caminho_redd, "redd.h5")

# VARIAVEL AUXILIAR
# Path dos arquivos relacionados as janelas
caminho_janelas = os.path.join(caminho_redd, "../../phd")
if not os.path.isdir(caminho_janelas):
    os.makedirs(caminho_janelas)

# Dados

In [3]:
# Checar se df dos resultados foi carregado
if not 'df_resultados' in locals():
    df_resultados = pd.read_csv(os.path.join(os.path.join(caminho_dados, "17"), "resultados_taxa-amostragem_janelas.csv"), index_col=[0])
df_resultados

,carga,taxa_amostragem,janela,loss,acuracia,precisao,recall,f1,f1_macro
0,dish_washer - 9,2,1080,0.03,97.00,0.00,0.00,0.00,49.24
1,fridge - 7,2,1080,0.00,100.00,100.00,100.00,100.00,100.00
2,microwave - 16,2,1080,0.06,94.00,50.00,16.67,25.00,60.94
3,washer_dryer - 13,2,1080,0.08,92.00,33.33,60.00,42.86,69.28
4,washer_dryer - 14,2,1080,0.08,92.00,33.33,33.33,33.33,64.54
...,...,...,...,...,...,...,...,...,...
150,dish_washer - 9,5,90,0.02,97.69,0.00,0.00,0.00,49.42
151,fridge - 7,5,90,0.21,78.99,82.66,80.87,81.75,78.50
152,microwave - 16,5,90,0.02,98.11,0.00,0.00,0.00,49.52
153,washer_dryer - 13,5,90,0.01,98.53,72.22,86.67,78.79,89.01


# Análise

## Melhor Taxa Amostral global e relação de melhores Janelas por Aparelho

In [4]:
# Verificar o desempenho GERAL (todos as janelas/aparelhos) por taxa amostral
# (premissa basica inicial pro sistema de NILM)
df_resultados.groupby("taxa_amostragem").agg({
    "acuracia":["count","mean", "std","min","max"],
    "f1_macro":["count","mean", "std","min","max"]
})

acuracia                                     f1_macro  \
                   count       mean       std    min     max    count   
taxa_amostragem                                                         
2                     45  94.357333  7.274399  71.67  100.00       45   
3                     45  94.198667  6.175332  73.59  100.00       45   
4                     40  93.646250  6.933205  73.01  100.00       40   
5                     25  94.402800  6.950410  75.44   99.37       25   

                                                      
                      mean        std    min     max  
taxa_amostragem                                       
2                68.275556  15.679079  48.28  100.00  
3                66.889556  15.864066  47.09  100.00  
4                67.329500  17.355146  45.95  100.00  
5                67.888800  15.919910  48.50   94.26

**NOTA:** A melhor Taxa Amostral global:

* Considerando `Acurácia`: 5 segundos (diferente do utilizado no estudo anterior - 3)
* Considerando `F1-score`: 2 segundos (adotado a partir deste estudo)

In [5]:
# Melhor taxa amostragem encontrada
melhor_taxa = 2

# Metrica utilizada para selecionar benchmark
METRICA_COMPARATIVA = {"metrica": "f1_macro", "rotulo": "F1-macro"}

In [6]:
# Verificar as melhores janelas obtidas para a taxa amostral + benchmark para pesquisa
df_melhores_janelas = pd.DataFrame(data={},columns=df_resultados.columns)
for carga in df_resultados["carga"].unique():
    print(carga)
    df = df_resultados.loc[
        (df_resultados["taxa_amostragem"]==melhor_taxa) & \
            (df_resultados["carga"]==carga),
        :
    ]
    df.reset_index(inplace=True, drop=True)

    df.sort_values(METRICA_COMPARATIVA["metrica"], ascending=False, inplace=True)
    melhor_score = df.iloc[0]
    df_melhores_janelas = df_melhores_janelas.append(melhor_score, ignore_index=True)

# Melhores janelas por aparelho (taxa fixa = 8)
df_melhores_janelas

dish_washer - 9
fridge - 7
microwave - 16
washer_dryer - 13
washer_dryer - 14


,carga,taxa_amostragem,janela,loss,acuracia,precisao,recall,f1,f1_macro
0,dish_washer - 9,2,720,0.05,95.33,20.00,25.00,22.22,59.91
1,fridge - 7,2,1080,0.00,100.00,100.00,100.00,100.00,100.00
2,microwave - 16,2,900,0.04,95.83,66.67,33.33,44.44,71.14
3,washer_dryer - 13,2,60,0.00,99.89,100.00,95.74,97.83,98.88
4,washer_dryer - 14,2,60,0.02,98.03,97.37,52.11,67.89,83.44


In [12]:
# Persistindo dados das melhores combinções de taxa e janela por aparelho
df_melhores_janelas.to_csv(os.path.join(caminho_dados_notebook, f"melhores_taxa{melhor_taxa}_janela_aparelhos.csv"))

In [13]:
from itertools import repeat
comparativo = {"taxa": []}
comparativo

{'taxa': []}

In [14]:
# Resumo por metrica
comparativo["taxa"].append("fixa")
for metrica in ["acuracia","precisao","recall","f1","f1_macro","loss"]:
    print("*", metrica+":")
    print("---")
    print("mean: {:.3f}".format(df_melhores_janelas[metrica].mean()) )
    print("std : {:.3f}".format(df_melhores_janelas[metrica].std()) )
    print("max : {:.3f}".format(df_melhores_janelas[metrica].max()) )
    print("min : {:.3f}".format(df_melhores_janelas[metrica].min()) )    
    print()
    
    if not metrica in comparativo:
        comparativo[metrica] = []
        
    comparativo[metrica].append(df_melhores_janelas[metrica].mean())
# pd.set_option('display.float_format', lambda x: '%.3f' % x)
# df_melhores_janelas.groupby(["taxa_amostragem"]).agg({
#     "acuracia": ["mean","std","min","max"],
#     "precisao": ["mean","std","min","max"],
#     "recall": ["mean","std","min","max"],
#     #"f1": ["mean","std","min","max"],
#     "f1_macro": ["mean","std","min","max"],
#     "loss": ["mean","std","min","max"],
# }).T

* acuracia:
---
mean: 97.816
std : 2.193
max : 100.000
min : 95.330

* precisao:
---
mean: 76.808
std : 34.744
max : 100.000
min : 20.000

* recall:
---
mean: 61.236
std : 34.886
max : 100.000
min : 25.000

* f1:
---
mean: 66.476
std : 33.738
max : 100.000
min : 22.220

* f1_macro:
---
mean: 82.674
std : 17.426
max : 100.000
min : 59.910

* loss:
---
mean: 0.022
std : 0.023
max : 0.050
min : 0.000



## Melhores combinações de Taxa Amostral e Janelas por Aparelho

In [15]:
# Verificar as melhores janelas obtidas para a taxa amostral + benchmark para pesquisa
df_melhores_taxas_janelas = pd.DataFrame(data={},columns=df_resultados.columns)
for carga in df_resultados["carga"].unique():
    df = df_resultados.loc[
        (df_resultados["carga"]==carga),
        :
    ]
    df.reset_index(inplace=True, drop=True)

    df.sort_values(METRICA_COMPARATIVA["metrica"], ascending=False, inplace=True)
    melhor_score = df.iloc[0]
    df_melhores_taxas_janelas = df_melhores_taxas_janelas.append(melhor_score, ignore_index=True)

# Melhores taxas e janelas por aparelho (taxa variavel)
df_melhores_taxas_janelas

,carga,taxa_amostragem,janela,loss,acuracia,precisao,recall,f1,f1_macro
0,dish_washer - 9,2,720,0.05,95.33,20.00,25.00,22.22,59.91
1,fridge - 7,2,1080,0.00,100.00,100.00,100.00,100.00,100.00
2,microwave - 16,2,900,0.04,95.83,66.67,33.33,44.44,71.14
3,washer_dryer - 13,2,60,0.00,99.89,100.00,95.74,97.83,98.88
4,washer_dryer - 14,3,360,0.02,97.99,100.00,55.56,71.43,85.19


In [16]:
# Resumo por metrica
comparativo["taxa"].append("variavel")
for metrica in ["acuracia","precisao","recall","f1","f1_macro","loss"]:
    print("*", metrica+":")
    print("---")
    print("mean: {:.3f}".format(df_melhores_taxas_janelas[metrica].mean()) )
    print("std : {:.3f}".format(df_melhores_taxas_janelas[metrica].std()) )
    print("max : {:.3f}".format(df_melhores_taxas_janelas[metrica].max()) )
    print("min : {:.3f}".format(df_melhores_taxas_janelas[metrica].min()) )    
    
    comparativo[metrica].append(df_melhores_taxas_janelas[metrica].mean())
    
    print()

* acuracia:
---
mean: 97.808
std : 2.192
max : 100.000
min : 95.330

* precisao:
---
mean: 77.334
std : 35.150
max : 100.000
min : 20.000

* recall:
---
mean: 61.926
std : 34.694
max : 100.000
min : 25.000

* f1:
---
mean: 67.184
std : 33.813
max : 100.000
min : 22.220

* f1_macro:
---
mean: 83.024
std : 17.463
max : 100.000
min : 59.910

* loss:
---
mean: 0.022
std : 0.023
max : 0.050
min : 0.000



In [17]:
# Persistindo dados das melhores combinções de taxa e janela por aparelho
df_melhores_taxas_janelas.to_csv(os.path.join(caminho_dados_notebook, "melhores_taxa_janela_aparelhos.csv"))

## Comparação: Taxa Fixa vs. Taxa Variável por Aparelho

In [18]:
df_comparativo = pd.DataFrame(comparativo)
df_comparativo

,taxa,acuracia,precisao,recall,f1,f1_macro,loss
0,fixa,97.816,76.808,61.236,66.476,82.674,0.022
1,variavel,97.808,77.334,61.926,67.184,83.024,0.022


In [19]:
df_comparativo.set_index("taxa").diff()

,acuracia,precisao,recall,f1,f1_macro,loss
taxa,,,,,,
fixa,NaN,NaN,NaN,NaN,NaN,NaN
variavel,-0.008,0.526,0.69,0.708,0.35,0.0


## Comparação: Diferenças Considerando F1-score (macro)

In [20]:
df_comparacao2 = pd.DataFrame(
    np.hstack([
        df_melhores_janelas[["taxa_amostragem","janela",METRICA_COMPARATIVA["metrica"]]].values,
        df_melhores_taxas_janelas[["taxa_amostragem","janela",METRICA_COMPARATIVA["metrica"]]].values,
        np.expand_dims(
            df_melhores_taxas_janelas[METRICA_COMPARATIVA["metrica"]].values - \
                df_melhores_janelas[METRICA_COMPARATIVA["metrica"]].values, 
            axis=1
        ),
    ]),  
    index=df_melhores_taxas_janelas["carga"].values, 
    columns=[
        ["Taxa Fixa", "Taxa Fixa", "Taxa Fixa",
         "Taxa Variável", "Taxa Variável", "Taxa Variável", 
         "Diferença"],
        ["Taxa","Janela", METRICA_COMPARATIVA["rotulo"], 
         "Taxa","Janela", METRICA_COMPARATIVA["rotulo"], 
         METRICA_COMPARATIVA["rotulo"]]
    ]
)
df_comparacao2

Taxa Fixa                 Taxa Variável                  \
                       Taxa Janela F1-macro          Taxa Janela F1-macro   
dish_washer - 9           2    720    59.91             2    720    59.91   
fridge - 7                2   1080      100             2   1080      100   
microwave - 16            2    900    71.14             2    900    71.14   
washer_dryer - 13         2     60    98.88             2     60    98.88   
washer_dryer - 14         2     60    83.44             3    360    85.19   

                  Diferença  
                   F1-macro  
dish_washer - 9           0  
fridge - 7                0  
microwave - 16            0  
washer_dryer - 13         0  
washer_dryer - 14      1.75

# Apenas aparelhos do paper IEEE PES GM

In [21]:
df_comparacao2.loc[ df_comparacao2.index.isin([
        "fridge - 7", "dish_washer - 9", "washer_dryer - 13", 
        "washer_dryer - 14", "microwave - 16"
    ]),:]

Taxa Fixa                 Taxa Variável                  \
                       Taxa Janela F1-macro          Taxa Janela F1-macro   
dish_washer - 9           2    720    59.91             2    720    59.91   
fridge - 7                2   1080      100             2   1080      100   
microwave - 16            2    900    71.14             2    900    71.14   
washer_dryer - 13         2     60    98.88             2     60    98.88   
washer_dryer - 14         2     60    83.44             3    360    85.19   

                  Diferença  
                   F1-macro  
dish_washer - 9           0  
fridge - 7                0  
microwave - 16            0  
washer_dryer - 13         0  
washer_dryer - 14      1.75

# Conclusões

1. A escolha de taxa amostral variável junto as janelas também variáveis por aparelho demonstrou ser útil para a melhoria dos classificadores, resultando em um `f1-macro ~3.75%` superior a abordagem considerando uma taxa fixa (hipótese inicial);

2. Quando comparado o resultado desta metodologia apenas nas cargas de maior consumo da residência 3 (consideradas no paper *IEEE PES GM 2020), observa-se ganhos relevantes, de até ~15.4% (wash dryer 1) em relação a hipótese de taxa fixa.

Sendo assim, observa-se a eficácia em adotar adotar uma metodologia híbrida para escolha de taxa amostral e janelas por aparelho, a qual deverá ser considerada para a continuidade da pesquisa.

# Fim.

In [22]:
%watermark -a "Diego Luiz Cavalca" -u -n -t -z -v -m -g

Author: Diego Luiz Cavalca

Last updated: Tue Mar 30 2021 08:15:02Hora oficial do Brasil

Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.21.0

Compiler    : MSC v.1928 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 158 Stepping 9, GenuineIntel
CPU cores   : 8
Architecture: 64bit

Git hash: 5725caa95e9d7f4b19a57eff5b998f1738bc40b4

